<p style="background-color:#D0D0FF; padding:10px;">This notebook presents solutions to the exercises in <a href="http://www.greenteapress.com/compmod/html/thinkcomplexity005.html" target="_blank">Chapter 4</a> of Allen B. Downey's <a href="http://www.greenteapress.com/compmod/" target="_blank">Think Complexity</a></p>

In [1]:
import itertools

# Provided classes (solution from Chapter 2)
class Vertex(object):
    """A Vertex is a node in a graph."""

    def __init__(self, label=''):
        self.label = label

    def __repr__(self):
        """Returns a string representation of this object that can
        be evaluated as a Python expression."""
        return 'Vertex(%s)' % repr(self.label)

    __str__ = __repr__
    """The str and repr forms of this object are the same."""


class Edge(tuple):
    """An Edge is a list of two vertices."""

    def __new__(cls, *vs):
        """The Edge constructor takes two vertices."""
        if len(vs) != 2:
            raise ValueError('Edges must connect exactly two vertices.')
        return tuple.__new__(cls, vs)

    def __repr__(self):
        """Return a string representation of this object that can
        be evaluated as a Python expression."""
        return 'Edge(%s, %s)' % (repr(self[0]), repr(self[1]))

    __str__ = __repr__
    """The str and repr forms of this object are the same."""


class Graph(dict):
    """A Graph is a dictionary of dictionaries.  The outer
    dictionary maps from a vertex to an inner dictionary.
    The inner dictionary maps from other vertices to edges.
    
    For vertices a and b, graph[a][b] maps
    to the edge that connects a->b, if it exists."""

    def __init__(self, vs=[], es=[]):
        """Creates a new graph.  
        vs: list of vertices;
        es: list of edges.
        """
        for v in vs:
            self.add_vertex(v)
            
        for e in es:
            self.add_edge(e)

    def add_vertex(self, v):
        """Add a vertex to the graph."""
        self[v] = {}

    def add_edge(self, e):
        """Adds and edge to the graph by adding an entry in both directions.

        If there is already an edge connecting these Vertices, the
        new edge replaces it.
        """
        v, w = e
        self[v][w] = e
        self[w][v] = e
        
    def has_key(self, k):
        print('Deprecated usage! For Python 3, use "<key> in <variable>" syntax.')
        return self.__contains__(k)
    
    def get_edge(self, v1, v2):
        '''Return the edge between two vertices if they exist'''
        if v1 not in self:
            return None
        elif v2 in self[v1]:
            return self[v1][v2]
        elif v2 not in self:
            # in case of one-way edges
            return None
        elif v1 not in self[v2]:
            return self[v2][v1]
        else:
            return None
    
    def remove_edge(self, e):
        '''Remove edge from graph'''
        v1, v2 = e
        deleted = ()
        if v1 not in self:
            pass
        elif v2 in self[v1]:
            del self[v1][v2]
            deleted = e

        # check for both directions
        if v2 not in self:
            return ()
        elif v1 in self[v2]:
            del self[v2][v1]
            deleted = e

        # return the deleted edge
        return deleted
    
    def vertices(self):
        '''return a list of vertices in a graph'''
        v = set(self.keys())
        for k in self.keys():
            v.update(self[k].keys())
        return list(v)
    
    def edges(self):
        '''return a list of unique edges in a graph'''
        e = set()
        for k in self.keys():
            e.update(self[k].values())
        return list(e)
    
    def out_vertices(self, v):
        '''return a list of vertices connected out from specified vertex'''
        v_out = set()
        if v in self:
            v_out.update(self[v].keys())
        return list(v_out)
    
    def out_edges(self, v):
        '''return a list of edges connected out from specified vertex'''
        e_out = set()
        if v in self:
            e_out.update(self[v].values())
        return list(e_out)

    def add_all_edges(self):
        '''Create edges between all vertices'''
        created=0
        for v1, v2 in itertools.combinations(self.keys(), 2):
            if v2 not in self[v1]:
                created += 1
                self.add_edge(Edge(v1, v2))
        return created
    
    def add_regular_edges(self, k):
        '''Add edges so that every vertex has the same degree k'''
        n = len(self)
        if self.edges():
            raise TypeError('Function requires an edgeless graph')
        elif n < k+1:
            raise ValueError('Maximum degree is 1 less than the number of vertices ({0:d})'.format(len(self)))
        elif k*n & 1:
            raise ValueError('Product of the degree and number of vertices ({0:d}) must be even'.format(len(self)))

        created=0
        if k == 0:
            return created

        vertices = self.vertices()
        m = int(k / 2)
        for i, v in enumerate(vertices):
            for j in range(m):
                self.add_edge(Edge(v, vertices[(i + j + 1) % n]))

            if k & 1:
                self.add_edge(Edge(v, vertices[(i + (n / 2)) % n]))
                
    def is_connected(self):
        '''returns True if the Graph is connected and False otherwise'''
        vertices = set(self.vertices())

        for v in self.keys():
            # Set of connections from vertex...
            s = set(self[v].keys())
            # ...plus the vertex itself...
            s.add(v)
            # ...should account for all of the vertices in the graph
            if vertices.difference(s):
                return False

        # Every node is connected to each other
        return True

In [2]:
import random

class RandomGraph(Graph):
    """An Erdos-Renyi random graph is a Graph where the probability of an edge between any two nodes is (p).

Instances are created as follows:

RandomGraph(list_of_vertices, list_of_edges)
RandomGraph(number_of_vertices, probability_of_edges)
    """
    def add_random_edges(self, p):
        '''adds edges at random so that the probability is p that there is an edge between any two nodes'''
        if self.edges():
            raise TypeError('Function requires an edgeless graph')
        
        random.seed(42) # no args to seed with current time
        created=0
        
        # itertools.combinations checks that v1 > v2, so this won't roll the dice twice
        for v1, v2 in itertools.combinations(self.keys(), 2):
            if random.random() <= p:
                created += 1
                self.add_edge(Edge(v1, v2))
        
        return created

def generateRandomGraph(n, p):
    """Generate a random graph with n vertices and probability of connection p"""
    g = RandomGraph([ Vertex(str(i)) for i in range(n) ], [])
    g.add_random_edges(p)
    return g

def testRandomGraph(n, p, min_tests=1000):
    '''Generate RandomGraph's and return the fraction of how many are connected'''
    connected = 0
    tests = min_tests
    for i in range(min_tests):
        if generateRandomGraph(n, p).is_connected():
            connected += 1
    
    return float(connected)/tests

def testConnectedness(n, places=4, verbose=True, min_tests=100):
    '''Simple search for p values where connectivity > 0% and < 100%'''
    start = 0
    stop = 1
    for digit in range(1,places+1):
        p_test = [ i * 10**(-1*digit) for i in range(int(start * 10**digit), int(stop * 10**digit + 1)) ]
        connected = [ (testRandomGraph(n,i,min_tests), i) for i in p_test ]
        start_idx = [ i[0] > 0 for i in connected ].index(True)-1
        stop_idx  = [ i[0] < 1 for i in connected ].index(False)
        start = connected[start_idx][1]
        stop  = connected[stop_idx][1]
        if verbose:
            s = 'Round {0:d}: f(n={3:d}, p={1:.'+'{0:d}'.format(places)+'f} to {2:.'+'{0:d}'.format(places)+'f}) = 0.0 - 1.0'
            print(s.format(digit, start, stop, n))
    
    #return final results
    return connected[start_idx:stop_idx+1]

<p style="background-color:#D0D0FF; padding:10px;">*Exercise 1:* Write an implementation of a FIFO using either a doubly-linked list or a circular buffer.</p>

In [3]:
from collections import deque

class FIFO(deque):
    def write(self, value):
        self.append(value)
    
    def read(self):
        return self.popleft()

f = FIFO('ab')
f.write('c')
f.write('d')
f.read(), f.read(), f.read()

('a', 'b', 'c')

<p style="background-color:#D0D0FF; padding:10px;">
<em>Exercise 4:</em> Create a file named SmallWorldGraph.py and define a class named SmallWorldGraph that inherits from RandomGraph.
<br><br>
If you did Exercise 2.4 you can use your own RandomGraph.py; otherwise you can download mine from thinkcomplex.com/RandomGraph.py.
<br><br>
1. Write a method called rewire that takes a probability p as a parameter and, starting with a regular graph, rewires the graph using Watts and Strogatz’s algorithm ([link](http://www.nature.com/nature/journal/v393/n6684/abs/393440a0.html)).
<br>
2. Write a method called clustering_coefficient that computes and returns the clustering coefficient as defined in the paper.
<br>
3. Make a graph that replicates the line marked C(p)/C(0) in Figure 2 of the paper. In other words, confirm that the clustering coefficient drops off slowly for small values of p.
</p>

In [4]:
def SmallWorldEdgeParams(k=5):
    if k == 0:
        raise StopIteration
    
    current = 0
    if k > 0:
        yield 0, 1, 1
        current += 1
    
    if k > 1:
        yield 1, 1, 2
        current += 1
    
    start=0
    distance=2
    round2=False
    while current < k:
        yield start, distance, current+1
        start += 1
        if (not round2) and (start == distance):
            current += 1
            round2 = True
        elif round2 and (start == distance*2):
            start = 0
            distance += 1
            current += 1
            round2 = False
            
for i, d, k in SmallWorldEdgeParams(5):
    print(i, d, k)    

0 1 1
1 1 2
0 2 3
1 2 3
2 2 4
3 2 4
0 3 5
1 3 5
2 3 5


In [5]:
def IterSmallWorldNodes(vertices='abcdefghijlmnop', offset=0, distance=2, less=True):
    n = len(vertices)
    step = distance*2
    i = offset
    if less: # don't wrap around
        fudge = distance
    else:
        fudge = 0
        
    while i+fudge < n:
        yield vertices[i], vertices[(i+distance) % n]
        i += step

for v1, v2 in IterSmallWorldNodes():#'abcdefghijlmnopr', offset=0, distance=3):
    print(v1, v2)

a c
e g
i l
n p


In [6]:
s = []
vertices='abcdefghijlmnopr'

for i in (0,1,2):
    for v1, v2 in IterSmallWorldNodes(vertices, i, 3):
        s.append(v1)
        s.append(v2)
        
print(vertices)
print(''.join(sorted(s)))

abcdefghijlmnopr
abcdefghijlmnr


In [7]:
from math  import log
from cmath import log as clog
from numpy import log as nplog

%timeit log(17)
print('')
%timeit clog(17)
print('')
%timeit nplog(17)

The slowest run took 38.96 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 180 ns per loop

The slowest run took 50.46 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 388 ns per loop

The slowest run took 16.03 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 1.05 µs per loop


In [8]:
def ln(x):
    return log(x).real

class SmallWorldGraph(RandomGraph):
    '''Generates a Small World Graph with edges on k nearest neighbors for n nodes'''
    def __init__(self, n=5, k=2):
        if not (n > k > ln(n) > 1):
            raise ValueError('Small World graphs must satistfy n > k > ln(n) > 1')
        vertices = [ Vertex(str(i)) for i in range(n) ]
        edges = []
        
        for i, d, current_k in SmallWorldEdgeParams(k):
            for v1, v2 in IterSmallWorldNodes(vertices, i, d):
                edges.append(Edge(v1, v2))
        
        RandomGraph.__init__(self, vertices, edges)
        
g = SmallWorldGraph(20, 4)
g

{Vertex('8'): {Vertex('9'): Edge(Vertex('8'), Vertex('9')),
  Vertex('7'): Edge(Vertex('7'), Vertex('8')),
  Vertex('6'): Edge(Vertex('6'), Vertex('8')),
  Vertex('10'): Edge(Vertex('8'), Vertex('10'))},
 Vertex('16'): {Vertex('18'): Edge(Vertex('16'), Vertex('18')),
  Vertex('17'): Edge(Vertex('16'), Vertex('17')),
  Vertex('15'): Edge(Vertex('15'), Vertex('16')),
  Vertex('14'): Edge(Vertex('14'), Vertex('16'))},
 Vertex('4'): {Vertex('6'): Edge(Vertex('4'), Vertex('6')),
  Vertex('5'): Edge(Vertex('4'), Vertex('5')),
  Vertex('3'): Edge(Vertex('3'), Vertex('4')),
  Vertex('2'): Edge(Vertex('2'), Vertex('4'))},
 Vertex('9'): {Vertex('8'): Edge(Vertex('8'), Vertex('9')),
  Vertex('7'): Edge(Vertex('7'), Vertex('9')),
  Vertex('11'): Edge(Vertex('9'), Vertex('11')),
  Vertex('10'): Edge(Vertex('9'), Vertex('10'))},
 Vertex('18'): {Vertex('19'): Edge(Vertex('18'), Vertex('19')),
  Vertex('17'): Edge(Vertex('17'), Vertex('18')),
  Vertex('16'): Edge(Vertex('16'), Vertex('18'))},
 Vertex(